<a href="https://colab.research.google.com/github/starlingomez/preworkmac/blob/master/Diciembre_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#clint capela

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [10.4], # Changed to list
    'avg_min_last5': [22.6], # Changed to list
    'opp_avg_reb_allowed': [46.2], # Changed to list
    'player_avg_reb_vs_opp': [4], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 76.1969 - mae: 8.2059 - val_loss: 65.1779 - val_mae: 7.5928
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 50.5564 - mae: 6.4846 - val_loss: 36.0507 - val_mae: 5.3637
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 24.4454 - mae: 4.2142 - val_loss: 13.2683 - val_mae: 3.0964
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.5996 - mae: 2.5357 - val_loss: 5.5043 - val_mae: 1.8647
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.1064 - mae: 1.9727 - val_loss: 4.4297 - val_mae: 1.5854
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8665 - mae: 1.9454 - val_loss: 4.2782 - val_mae: 1.5376
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0784 - mae: 1.7844 - val_loss: 4.3062 - val_mae: 1.5632
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1311 - mae: 1.8052 - val_loss: 4.2116 - val_mae: 1.5377
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss

#jalen jonshon

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [9], # Changed to list
    'avg_min_last5': [36.4], # Changed to list
    'opp_avg_reb_allowed': [46.2], # Changed to list
    'player_avg_reb_vs_opp': [9], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 81.0570 - mae: 8.5394 - val_loss: 72.4610 - val_mae: 8.1592
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 57.1126 - mae: 7.0637 - val_loss: 46.8610 - val_mae: 6.4553
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.1989 - mae: 5.2965 - val_loss: 20.9049 - val_mae: 4.1779
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 14.8800 - mae: 3.1892 - val_loss: 6.6736 - val_mae: 2.1376
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.1147 - mae: 1.9884 - val_loss: 4.2852 - val_mae: 1.5890
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7185 - mae: 1.9088 - val_loss: 4.3576 - val_mae: 1.6318
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3560 - mae: 1.8396 - val_loss: 4.3177 - val_mae: 1.6227
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0514 - mae: 1.7765 - val_loss: 4.3228 - val_mae: 1.6268
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9950 - 

#okongwo

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [7.2], # Changed to list
    'avg_min_last5': [24.5], # Changed to list
    'opp_avg_reb_allowed': [46.2], # Changed to list
    'player_avg_reb_vs_opp': [9], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 81.0570 - mae: 8.5394 - val_loss: 72.4610 - val_mae: 8.1592
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 57.1126 - mae: 7.0637 - val_loss: 46.8610 - val_mae: 6.4553
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.1989 - mae: 5.2965 - val_loss: 20.9049 - val_mae: 4.1779
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 14.8800 - mae: 3.1892 - val_loss: 6.6736 - val_mae: 2.1376
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.1147 - mae: 1.9884 - val_loss: 4.2852 - val_mae: 1.5890
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7185 - mae: 1.9088 - val_loss: 4.3576 - val_mae: 1.6318
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3560 - mae: 1.8396 - val_loss: 4.3177 - val_mae: 1.6227
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0514 - mae: 1.7765 - val_loss: 4.3228 - val_mae: 1.6268
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9950 - 

#jason tatum

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [9.6], # Changed to list
    'avg_min_last5': [35.9], # Changed to list
    'opp_avg_reb_allowed': [46.2], # Changed to list
    'player_avg_reb_vs_opp': [8.7], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 75.7025 - mae: 8.2387 - val_loss: 65.7218 - val_mae: 7.7019
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51.6438 - mae: 6.6352 - val_loss: 37.6600 - val_mae: 5.6592
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27.5972 - mae: 4.6721 - val_loss: 12.5984 - val_mae: 3.1111
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.4233 - mae: 2.3950 - val_loss: 5.3595 - val_mae: 1.8215
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9803 - mae: 1.7654 - val_loss: 4.8793 - val_mae: 1.6944
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0547 - mae: 1.7727 - val_loss: 4.8524 - val_mae: 1.7171
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.8761 - mae: 1.7518 - val_loss: 4.6823 - val_mae: 1.6706
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.7200 - mae: 1.7016 - val_loss: 4.6294 - val_mae: 1.6786
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.4319 - m

#rudy gobert

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [13.2], # Changed to list
    'avg_min_last5': [38.7], # Changed to list
    'opp_avg_reb_allowed': [43.6], # Changed to list
    'player_avg_reb_vs_opp': [14], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 79.7011 - mae: 8.4542 - val_loss: 71.7385 - val_mae: 8.0756
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 57.6003 - mae: 7.0257 - val_loss: 45.5846 - val_mae: 6.2405
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.5841 - mae: 4.9355 - val_loss: 19.3621 - val_mae: 3.8141
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.3503 - mae: 2.7730 - val_loss: 6.6434 - val_mae: 2.0653
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.3033 - mae: 2.0017 - val_loss: 4.7252 - val_mae: 1.6731
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3761 - mae: 1.8587 - val_loss: 4.4911 - val_mae: 1.6327
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.2952 - mae: 1.8141 - val_loss: 4.4766 - val_mae: 1.6293
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.1994 - mae: 1.8091 - val_loss: 4.3743 - val_mae: 1.5943
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - los

#nic claxton

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [6.8], # Changed to list
    'avg_min_last5': [24.7], # Changed to list
    'opp_avg_reb_allowed': [46.3], # Changed to list
    'player_avg_reb_vs_opp': [9], # Changed to list
    'home_indicator': [0], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 82.8151 - mae: 8.6127 - val_loss: 79.3751 - val_mae: 8.5113
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 62.6353 - mae: 7.3880 - val_loss: 57.8862 - val_mae: 7.1328
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 43.7818 - mae: 6.0154 - val_loss: 31.2268 - val_mae: 5.0286
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 20.6484 - mae: 3.8882 - val_loss: 11.6308 - val_mae: 2.8416
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.0994 - mae: 2.2579 - val_loss: 5.9687 - val_mae: 1.9596
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8862 - mae: 1.8838 - val_loss: 5.0323 - val_mae: 1.7608
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0851 - mae: 1.8025 - val_loss: 4.9042 - val_mae: 1.7328
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.8147 - mae: 1.7531 - val_loss: 4.7842 - val_mae: 1.7005
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.1702 -

#cameron jonhson

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [4.8], # Changed to list
    'avg_min_last5': [28.6], # Changed to list
    'opp_avg_reb_allowed': [46.3], # Changed to list
    'player_avg_reb_vs_opp': [6], # Changed to list
    'home_indicator': [0], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 78.8872 - mae: 8.4130 - val_loss: 74.2888 - val_mae: 8.1964
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 59.5055 - mae: 7.1945 - val_loss: 48.0885 - val_mae: 6.4103
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.4854 - mae: 5.2059 - val_loss: 21.4794 - val_mae: 4.0785
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 13.3731 - mae: 3.0054 - val_loss: 8.0999 - val_mae: 2.2981
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.2344 - mae: 2.0061 - val_loss: 5.3648 - val_mae: 1.8221
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7916 - mae: 1.9446 - val_loss: 4.9879 - val_mae: 1.7582
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.2755 - mae: 1.8389 - val_loss: 4.7975 - val_mae: 1.7131
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0214 - mae: 1.8038 - val_loss: 4.7634 - val_mae: 1.7087
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.4184 - m

#nikola vucevic

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [11], # Changed to list
    'avg_min_last5': [30.8], # Changed to list
    'opp_avg_reb_allowed': [42.7], # Changed to list
    'player_avg_reb_vs_opp': [11], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 83.4245 - mae: 8.6764 - val_loss: 76.5744 - val_mae: 8.3531
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 61.2758 - mae: 7.2800 - val_loss: 50.6552 - val_mae: 6.6210
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 36.0718 - mae: 5.3544 - val_loss: 22.3704 - val_mae: 4.1775
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.5578 - mae: 3.1533 - val_loss: 7.5601 - val_mae: 2.2717
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.8010 - mae: 2.1038 - val_loss: 4.9726 - val_mae: 1.7570
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8058 - mae: 1.9402 - val_loss: 4.6511 - val_mae: 1.6749
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0657 - mae: 1.8054 - val_loss: 4.5911 - val_mae: 1.6708
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9631 - mae: 1.7821 - val_loss: 4.5217 - val_mae: 1.6454
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - los

#josh gidey

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [5.4], # Changed to list
    'avg_min_last5': [25.5], # Changed to list
    'opp_avg_reb_allowed': [42.7], # Changed to list
    'player_avg_reb_vs_opp': [6], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 75.0906 - mae: 8.2059 - val_loss: 68.6400 - val_mae: 7.8821
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 53.3551 - mae: 6.7937 - val_loss: 38.4075 - val_mae: 5.7263
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25.8466 - mae: 4.4285 - val_loss: 12.6626 - val_mae: 3.0586
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.4311 - mae: 2.3521 - val_loss: 5.7943 - val_mae: 1.8760
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.9267 - mae: 1.9159 - val_loss: 5.1649 - val_mae: 1.7746
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.2837 - mae: 1.8014 - val_loss: 5.1307 - val_mae: 1.7958
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0061 - mae: 1.7546 - val_loss: 4.8968 - val_mae: 1.7261
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.5677 - mae: 1.7231 - val_loss: 4.8309 - val_mae: 1.7174
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.8542 - m